# Techstars Timesheet Data Prep

## Preliminary

In [103]:
# Load required packages
import pandas as pd
import numpy as np

## Data Import

In [104]:
# Import the raw data
df = pd.read_csv("raw_data/gcal_export_v01.csv")

## Data Cleaning

In [105]:
# Clean a mispelled entry in the data
df.Title = df.Title.replace('Technical Work\r',"Technical Work")

# Rename a variable from Title to Activity to make it more descriptive
df.rename(columns={'Title': 'Activity'}, inplace=True)

# Rename entries from Meeting to Meeting/Networking to make it more descriptive
df.Activity = df.Activity.replace('Meetings',"Discussion")

In [106]:
# Convert time format (4:30) into a numeric format (4.5)

# Create an empty list
duration_t = []

# For each row in df.Duration,
for row in df.Duration:
    # Create a variable called time that is row, but split along any ':'
    time = row.split(":")
    # Then append the first element of time (hours), and the second element of time divided by 60 (minutes)
    duration_t.append(int(time[0])+int(time[1])/60)
    
# Create a pandas column from the empty    
df['Duration'] = duration_t

In [107]:
# Create a variable for the date (removing the hour or minute)

# Create an empty list
date = []

# For each row in df.Start,
for row in df.Start:
    # Then convert row to a string, split it along the space, and append the first element of that split.
    date.append(str(row).split(' ')[0])

# Create a column from the list
df['Date'] = date

In [108]:
# Let's take a peak at the data now
df[df['Date'] == '11/2/15']

,Activity,Start,End,Duration,Date
0,Sleep,11/2/15 0:00,11/2/15 4:30,4.5,11/2/15
1,Personal,11/2/15 4:30,11/2/15 5:30,1.0,11/2/15
2,Non-Technical Work,11/2/15 5:30,11/2/15 8:00,2.5,11/2/15
3,Discussion,11/2/15 8:00,11/2/15 14:30,6.5,11/2/15
4,Non-Technical Work,11/2/15 14:30,11/2/15 16:00,1.5,11/2/15
5,Discussion,11/2/15 16:00,11/2/15 22:30,6.5,11/2/15
6,Non-Technical Work,11/2/15 22:30,11/3/15 0:00,1.5,11/2/15


In [109]:
# Save the dataframe as a csv file, without an index column
df.to_csv('clean_data/activity_data.csv', index=False)

## Create Interesting Subsets

### Create the main data

In [110]:
# Create a dataframe by copying the df dataframe
df_daily = df[:]
# Convert the Date column to a time data type
df_daily['Date'] = pd.to_datetime(df_daily['Date'])
# Make the Date column the dataframe's index
df_daily.index = df_daily['Date']
# Group the data by Activity and organize by day
df_daily = df_daily.groupby('Activity').resample('D', sum)
# Fill in missing values with "0" (we do this because we know that if an activity has a missing value for a day, it means we didn't do it, thus spent 0 hours on it)
df_daily = df_daily.fillna("0")

# Save as a csv file with an index column
df_daily.to_csv('clean_data/daily_data.csv', index=True)

### Dividing the time into Techstar's Three Stages

#### Stage 1: Planning

In [111]:
# Create a start date and an end date
planning_start = pd.Timestamp('2015-11-02')
planning_end = pd.Timestamp('2015-12-02')

# Create a dataframe by copying the df dataframe
df_planning = df[:]
# Convert the Date column to a time data type
df_planning['Date'] = pd.to_datetime(df_planning['Date'])
# Make the Date column the dataframe's index
df_planning.index = df_planning['Date']
# Slice the data by the start date and end date
df_planning = df_planning[planning_start:planning_end ]
# Group the data by Activity and organize by day
df_planning = df_planning.groupby('Activity').resample('D', sum)
# Fill in missing values with "0" (we do this because we know that if an activity has a missing value for a day, it means we didn't do it, thus spent 0 hours on it)
df_planning = df_planning.fillna("0")

# Save as a csv file with an index column
df_planning.to_csv('clean_data/planning_stage_data.csv', index=True)

#### Stage 2: Building

In [112]:
# Create a start date and an end date
building_start = pd.Timestamp('2015-12-03')
building_end = pd.Timestamp('2016-01-10')

# Create a dataframe by copying the df dataframe
df_building = df[:]
# Convert the Date column to a time data type
df_building['Date'] = pd.to_datetime(df_building['Date'])
# Make the Date column the dataframe's index
df_building.index = df_building['Date']
# Slice the data by the start date and end date
df_building = df_building[building_start:building_end ]
# Group the data by Activity and organize by day
df_building = df_building.groupby('Activity').resample('D', sum)
# Fill in missing values with "0" (we do this because we know that if an activity has a missing value for a day, it means we didn't do it, thus spent 0 hours on it)
df_building = df_building.fillna("0")

# Save as a csv file with an index column
df_building.to_csv('clean_data/building_stage_data.csv', index=True)

#### Stage 3: Growing

In [113]:
# Create a start date and an end date
growing_start = pd.Timestamp('2016-01-10')
growing_end = pd.Timestamp('2016-01-24')

# Create a dataframe by copying the df dataframe
df_growing = df[:]
df_growing['Date'] = pd.to_datetime(df_growing['Date'])
# Make the Date column the dataframe's index
df_growing.index = df_growing['Date']
# Slice the data by the start date and end date
df_growing = df_growing[growing_start:growing_end ]
# Group the data by Activity and organize by day
df_growing = df_growing.groupby('Activity').resample('D', sum)
# Fill in missing values with "0" (we do this because we know that if an activity has a missing value for a day, it means we didn't do it, thus spent 0 hours on it)
df_growing = df_growing.fillna("0")

# Save as a csv file with an index column
df_growing.to_csv('clean_data/growing_data.csv', index=True)

### Creating data for major holidays

In [114]:
# Create a dataframe called thanksgiving which just contains activities on that day
thanksgiving = df_daily.query('Date == "2015-11-26"')
# Create a new column of the dataframe containing the name of that holiday
thanksgiving['Holiday'] = pd.Series(['Thanksgiving']*len(thanksgiving), index=thanksgiving.index)

/Users/chrisralbon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [115]:
# Create a dataframe called christmas which just contains activities on that day
christmas = df_daily.query('Date == "2015-12-25"')
# Create a new column of the dataframe containing the name of that holiday
christmas['Holiday'] = pd.Series(['Christmas']*len(christmas), index=christmas.index)

/Users/chrisralbon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [116]:
# Create a dataframe called new_years_ever which just contains activities on that day
new_years_eve = df_daily.query('Date == "2015-12-31"')
# Create a new column of the dataframe containing the name of that holiday
new_years_eve['Holiday'] = pd.Series(['New Years Eve']*len(new_years_eve), index=new_years_eve.index)

/Users/chrisralbon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [117]:
# Merge all three holiday dataframes together into one dataframe
df_holidays = pd.concat([thanksgiving ,christmas, new_years_eve])

In [118]:
# Save as a csv file with an index column
df_holidays.to_csv('clean_data/holiday_data.csv', index=True)

## Garage Pile

In [119]:
# This is a clever bit of code that tests to see if an activity doesnt exist each day and if it doesn't creates a new entry in the dataframe with that activity, but with 0 hours for duration.

# for date in df.Date.unique():
#
#    if any(df[df['Date'] == date].Activity == 'Sleep') == False:
#        df_sleep = pd.DataFrame([['Sleep', np.nan, np.nan, 0, date]], columns=['Activity', 'Start', 'End', 'Duration', 'Date'])
#        df = df.append(df_sleep)
#
#    if any(df[df['Date'] == date].Activity == 'Exercise') == False:
#        df_exercise = pd.DataFrame([['Exercise', np.nan, np.nan, 0, date]], columns=['Activity', 'Start', 'End', 'Duration', 'Date'])
#        df = df.append(df_exercise)
#
#    if any(df[df['Date'] == date].Activity == 'Personal') == False:
#        df_personal = pd.DataFrame([['Personal', np.nan, np.nan, 0, date]], columns=['Activity', 'Start', 'End', 'Duration', 'Date'])
#        df = df.append(df_personal)
#
#    if any(df[df['Date'] == date].Activity == 'Non-Technical Work') == False:
#        df_nontechnicalwork = pd.DataFrame([['Non-Technical Work', np.nan, np.nan, 0, date]], columns=['Activity', 'Start', 'End', 'Duration', 'Date'])
#        df = df.append(df_nontechnicalwork)
#
#    if any(df[df['Date'] == date].Activity == 'Technical Work') == False:
#        df_technicalwork = pd.DataFrame([['Technical Work', np.nan, np.nan, 0, date]], columns=['Activity', 'Start', 'End', 'Duration', 'Date'])
#        df = df.append(df_technicalwork)
#
#    if any(df[df['Date'] == date].Activity == 'Meetings') == False:
#        df_meetings = pd.DataFrame([['Meetings', np.nan, np.nan, 0, date]], columns=['Activity', 'Start', 'End', 'Duration', 'Date'])
#        df = df.append(df_technicalwork)
#
#    if any(df[df['Date'] == date].Activity == 'Travel') == False:
#        df_travel = pd.DataFrame([['Travel', np.nan, np.nan, 0, date]], columns=['Activity', 'Start', 'End', 'Duration', 'Date'])
#        df = df.append(df_travel)